# Uso de Scrapping para Acceder a Publicaciones en Mercado Libre

Veamos cómo trabajar para obtener datos de publicaciones de Mercado Libre, dada una determinada búsqueda

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

## Busqueda de precios y productos

En este caso, vamos a saltear la parte de cómo interactuar con la página para generar el url de la búsqueda y partiremos directamente con el url dado.

Buscaremos publicaciones de heladeras...suponiendo que queremos hacer un estudio de mercado sobre heladeras, precios y prestaciones.

In [ ]:
url = 'https://listado.mercadolibre.com.ar/electrodomesticos/heladeras-freezers/heladeras/heladera#D[A:heladera,on]'
search = requests.get(url)
search_parseada = bs(search.content, 'html.parser')

Inspeccionando el html de la página, podemos observar cómo se listan los distintos productos:



In [ ]:
lista_productos = search_parseada.findAll(name = 'li', attrs = {'class' : 'ui-search-layout__item'}) # Obtebemos todos los productos de la página

In [ ]:
len(lista_productos)

52

Vemos un problema si queremos hacer un estudio más profundo... salvo el título de la publicación y el precio, en estos tags no aparece mucha más información.

Podríamos tratar de sacar info útil del título, pero teniendo en cuenta que cada producto puede tener una forma de nombrar distitna, puede ser polémico.

Accedamos entonces compulsivamente a los precios de los productos

In [ ]:
nombre = lista_productos[0].find(name = 'h2', attrs = {'class' : 'ui-search-item__title'})
precio = lista_productos[0].find(name = 'span', attrs = {'class' : 'price-tag-amount'})
print(precio.prettify())
print(f'El precio del producto \033[1m{nombre.text}\033[0m es: {precio.text}')

<span aria-hidden="true" class="price-tag-amount">
 <span class="price-tag-symbol">
  $
 </span>
 <span class="price-tag-fraction">
  92.799
 </span>
</span>
El precio del producto Heladera Patrick HPK151M11 black steel con freezer 388L 220V es: $92.799


'$'

Ahora sí, vamos con todos los productos de la página:


In [ ]:
lista_precios = [p.find(name = 'span', attrs = {'class' : 'price-tag-amount'}).text.replace('$','').replace('.','') for p in lista_productos]
lista_precios

['92799',
 '91399',
 '47499',
 '124999',
 '90999',
 '45000',
 '73199',
 '195199',
 '73199',
 '49900',
 '64999',
 '52100',
 '86999',
 '175999',
 '68199',
 '280499',
 '57999',
 '47399',
 '188999',
 '102899',
 '47900',
 '45799',
 '44299',
 '256999',
 '37349',
 '39200',
 '169999',
 '74599',
 '51900',
 '54999',
 '68900',
 '64599',
 '116000',
 '114066',
 '74300',
 '77899',
 '53700',
 '40959',
 '58699',
 '163999',
 '104899',
 '42999',
 '107699',
 '183499',
 '79976',
 '41889',
 '46499',
 '171999',
 '64999',
 '90999',
 '68199',
 '56590']

Pero, como ya vimos, estos no son todos los resultados. Cómo paginamos con mercado libre? Podemos modificar el url de la siguiente forma:


In [ ]:
url_pagina_1 = 'https://listado.mercadolibre.com.ar/electrodomesticos/heladeras-freezers/heladeras/heladera_Desde_51'
url_pagina_2 = 'https://listado.mercadolibre.com.ar/electrodomesticos/heladeras-freezers/heladeras/heladera_Desde_101'
# y así, parece ser que hay que modificar ligeramente el url previo a ir agregando el 51, 101, 151, y así, sumando 50.

## Análisis de un Procuto Particular

Veamos ahora cómo hacemos para obtener toda la info de un producto determinado

In [ ]:
# Veamos la siguiente heladera (la que tengo je)
url_producto = 'https://www.mercadolibre.com.ar/heladera-gafa-hgf387af-blanca-con-freezer-374l-220v/p/MLA15231333?pdp_filters=category:MLA398582#searchVariation=MLA15231333&position=1&search_layout=stack&type=product&tracking_id=87d29059-3596-4c2d-991a-07564a7235aa'

search_producto = requests.get(url_producto)
search_producto_parseada = bs(search_producto.content, 'html.parser')

In [ ]:
# Busquemos las características que aparecen bajo el título "Lo que tenés que saber de este producto"
lo_que_hay_que_saber = search_producto_parseada.find(name = 'ul', attrs = {'class' : 'ui-vpp-highlighted-specs__features-list'})

In [ ]:
print(f'Lo que hay que saber del producto \033[1m{search_producto_parseada.find(name = "h1").text}\033[0m son:')
print('\n'.join([caract.text for caract in lo_que_hay_que_saber.findChildren(name = 'li')]))

Lo que hay que saber del producto Heladera Gafa HGF387AF blanca con freezer 374L 220V son:
Modelo detallado: HGF387AFB.
Tipo de deshielo: cycle defrost.
Capacidad de 374 litros.
Con freezer superior.
Eficiencia energética A.
Cuenta con porta huevos.
Dimensiones: 61.4 cm de ancho, 180 cm de alto y 62.1 cm de profundidad.


In [ ]:
caracteristicas_producto = search_producto_parseada.Modelo(name = 'tbody', attrs = {'class' : 'andes-table__body'})

In [ ]:
print(f'Las características de \033[1m{search_producto_parseada.find(name = "h1").text}\033[0m son:')
for child in caracteristicas_producto.children:
    print(child.th.text, child.td.text)

Las características de Heladera Gafa HGF387AF blanca con freezer 374L 220V son:
Marca Gafa
Modelo HGF387AF
Color Blanco
Voltaje 220V
Modelo detallado HGF387AFB
